#### 0501_Lab  
1. Cleanse categorical data in the Student table in BTech22_\<your name\>.sqlite.   

In [29]:
import pandas as pd
import os
import sqlite3 as lite
from glob import glob

dbpath = 'BTech22_reethurgt.sqlite'
con = lite.connect(dbpath)

In [30]:
df=pd.read_excel("/Users/reethu/coding/college/DBMS/Data/BTech22 Data.xlsx")
df

,USN,Name,Email,Section,Sex,DoB,Height_cm,Weight_Kg,Home_State,Home_Town,Mother_Tongue,Elective,Clubs
0,1RVU22CSE001,SHREE VYSHNAVI A,shreevyshnavia,B,Female,14/04/2005,158,56,Karnataka,Mysore,Telugu,Photography,Generative AI
1,1RVU22CSE002,AARON ARTHUR ELISHA,aarone,A,Male,2003-04-02 00:00:00,183,76,NaN,NaN,English,Community service,NaN
2,1RVU22CSE003,ABHISHEK,abhishek,A,Male,2003-08-05 00:00:00,181,56,Karnataka,Bidar,Marathi,Photography,NaN
3,1RVU22CSE004,ABRAR HUSSAIN,abrarh,B,Male,21/11/2003,182,69,Jammu,Bathindi,Urdu,Drumming,E-Cell
4,1RVU22CSE005,ADITI A,aditia,B,Female,13-3-2004,157,NaN,Karnataka,Ballari,Kannada,Photography,"Generative AI, Expressions, Kalakriti"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1RVU22CSE189,VIBHU POTTEKULA,vibhup,C,Male,18/03/04,172,56,Karnataka,Bengaluru,Kannada,Photography,Viksha
188,1RVU22CSE190,VIJAY SHARMITH,vijays,B,Male,2004-05-04 00:00:00,176,55,Karnataka,Bengaluru,Tamil,"Success, Happiness and Leadership","Cybersecurity, Neural Nexus club"
189,1RVU22CSE191,VIKASH KASHYAP,vikashk,A,Male,13/05/2003,178,85,Karnataka,Banglore,Kannada,Drumming,NaN
190,1RVU22CSE192,VISMITHA PRIYANKA,vismithap,A,Female,25/06/2004,160,54,Karnataka,Bangalore,kannada,Drumming,"cybersecurity, Neural Nexus ,Viksha"


In [31]:
con.execute("CREATE TABLE Students(USN, Name, Email, Section, Sex, DoB, Height_cm, Weight_Kg, Home_State, Home_Town, Mother_Tongue, Elective, Clubs, PRIMARY KEY (USN))")

In [32]:
df.to_sql('Students', con, index=False, if_exists='append')

192

In [33]:
con.execute("SELECT * FROM sqlite_master").fetchall()

[('table',
  'Students',
  'Students',
  2,
  'CREATE TABLE Students(USN, Name, Email, Section, Sex, DoB, Height_cm, Weight_Kg, Home_State, Home_Town, Mother_Tongue, Elective, Clubs, PRIMARY KEY (USN))'),
 ('index', 'sqlite_autoindex_Students_1', 'Students', 3, None)]

In [34]:
con.execute("CREATE TABLE Students_temp AS SELECT * FROM Students")
con.commit();

In [35]:
pd.read_sql("SELECT COUNT(*) FROM Students_temp", con)

,COUNT(*)
0,192


In [36]:
pd.read_sql("PRAGMA table_info(Students_temp)", con)

,cid,name,type,notnull,dflt_value,pk
0,0,USN,,0,None,0
1,1,Name,,0,None,0
2,2,Email,,0,None,0
3,3,Section,,0,None,0
4,4,Sex,,0,None,0
5,5,DoB,,0,None,0
6,6,Height_cm,,0,None,0
7,7,Weight_Kg,,0,None,0
8,8,Home_State,,0,None,0
9,9,Home_Town,,0,None,0


In [37]:
con.execute("SELECT DISTINCT Home_State FROM Students_temp").fetchall()

[('Karnataka ',),
 (None,),
 ('Karnataka',),
 ('Jammu',),
 ('Haryana',),
 ('karnataka ',),
 ('Delhi',),
 ('karnataka',),
 ('West Bengal',),
 ('Andhra Pradesh',),
 ('Telangana',),
 ('Punjab',),
 ('Goa',),
 ('Uttar Pradesh',),
 ('Rajasthan ',),
 ('Tamil Nadu',),
 ('Bihar',),
 ('Nil ',),
 ('Andhra Pradesh ',),
 ('Uttar Pradesh ',),
 ('KARNATAKA',),
 ('karntaka',),
 ('Gujarat',),
 ('Jharkhand',),
 ('-',),
 ('Rajasthan',),
 ('KARnataka',),
 ('Punjab ',),
 ('Karantaka',)]

In [38]:
query1 = """
    UPDATE Students
    SET Home_State = TRIM(Home_State)
"""
query2 = """
    UPDATE Students
    SET Home_State = 'Karnataka'
    WHERE Home_State IN
    ('karnataka','KARNATAKA','karntaka','KARnataka','Karantaka')
"""
query3 = """
    UPDATE Students
    SET Home_State = 'unknown'
    WHERE Home_State = 'Nil'
"""

In [39]:
con.execute(query1)
con.execute(query2)
con.execute(query3)
con.commit()

2. Create look-up tables (aka master tables) for states, towns, and languages.    
Insert data.     
See [RTO Codes](https://www.v3cars.com/car-guide/state-code-regional-transport-office-rto-india) and 
Census Codes for [States and Cities](https://censusindia.gov.in/nada/index.php/catalog/42648) and 
[Language Codes](https://censusindia.gov.in/nada/index.php/catalog/42561)  
Use standard codes or abbreviations as PK.  



In [28]:
con.execute("CREATE TABLE Master_Lang(Language TEXT PK, Code TEXT)")

In [40]:
con.execute("CREATE TABLE Master_State(State TEXT PK, Code TEXT)")

In [41]:
rows = [('Karnataka','KA'),
 ('Jammu','JK'),
 ('Haryana','HR'),
 ('Delhi','DL'),
 ('West Bengal','WB'),
 ('Andhra Pradesh','AP'),
 ('Telangana','TS'),
 ('Punjab','PB'),
 ('Goa','GA'),
 ('Uttar Pradesh','UP'),
 ('Rajasthan','RJ'),
 ('Tamil Nadu','TN'),
 ('Bihar','BR'),
 ('None','-1'),
 ('Gujarat','GJ'),
 ('Jharkhand','JH')]

In [42]:
con.executemany("INSERT INTO Master_State(State,Code) VALUES(?,?)",rows)

In [43]:
import pandas as pd
pd.read_sql("SELECT * FROM Master_State",con)

,State,Code
0,Karnataka,KA
1,Jammu,JK
2,Haryana,HR
3,Delhi,DL
4,West Bengal,WB
5,Andhra Pradesh,AP
6,Telangana,TS
7,Punjab,PB
8,Goa,GA
9,Uttar Pradesh,UP


In [44]:
sql = """
  UPDATE Students_temp
     SET Home_State = Master_State.code
     FROM Master_State
        WHERE Master_State.State = Students_temp.Home_State
"""
con.execute(sql)
con.commit()

In [45]:
pd.read_sql("SELECT Name,Section,Home_State FROM Students_temp LIMIT 10", con)

,Name,Section,Home_State
0,SHREE VYSHNAVI A,B,Karnataka
1,AARON ARTHUR ELISHA,A,None
2,ABHISHEK,A,KA
3,ABRAR HUSSAIN,B,JK
4,ADITI A,B,KA
5,ADITYA GARG,C,HR
6,ADITYA ADUKURI,B,KA
7,ADITYA KUSHAL,A,KA
8,ADITYA P,A,KA
9,ADITYA RAJU A,A,KA


In [46]:
sql = """
    ALTER TABLE Students_temp
    RENAME COLUMN Home_State TO state_id;
"""
con.execute(sql)
con.commit()

In [47]:
pd.read_sql("SELECT Name,Section,state_id FROM Students_temp LIMIT 10", con)

,Name,Section,state_id
0,SHREE VYSHNAVI A,B,Karnataka
1,AARON ARTHUR ELISHA,A,None
2,ABHISHEK,A,KA
3,ABRAR HUSSAIN,B,JK
4,ADITI A,B,KA
5,ADITYA GARG,C,HR
6,ADITYA ADUKURI,B,KA
7,ADITYA KUSHAL,A,KA
8,ADITYA P,A,KA
9,ADITYA RAJU A,A,KA


3. Create tables for electives and clubs.  
    Their PK may be auto-incremented.  
    Insert data.  